<a href="https://colab.research.google.com/github/wathsan11/Rextro-NILM-Project/blob/main/NILM_Multiclass_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

RANDOM_STATE = 99

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Machine Learning/NILM.csv')
df.head()

,voltage,current,active_power,reactive_power,apparent_power,power_factor,timestamp,reliable,device_id,load_type,probe_id,frequency,energy,temperature,bulb,fan,iron,labels
0,233.67,0.427060,84.82254,52.56829,99.79122,0.85,2025-12-09T19:02:50.725756,False,power_monitor_1,unknown,default,50,0,25,1,0,0,1
1,233.67,0.435269,86.45290,53.57870,101.70930,0.85,2025-12-09T19:02:53.781784,False,power_monitor_1,unknown,default,50,0,25,1,0,0,1
2,233.67,0.451382,89.65333,55.56215,105.47450,0.85,2025-12-09T19:02:56.810625,False,power_monitor_1,unknown,default,50,0,25,1,0,0,1
3,233.67,0.457015,90.77206,56.25548,106.79070,0.85,2025-12-09T19:02:59.867042,False,power_monitor_1,unknown,default,50,0,25,1,0,0,1
4,233.67,0.445174,88.42026,54.79794,104.02380,0.85,2025-12-09T19:03:02.925509,False,power_monitor_1,unknown,default,50,0,25,1,0,0,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3200 entries, 0 to 3199
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   voltage         3200 non-null   float64
 1   current         3200 non-null   float64
 2   active_power    3200 non-null   float64
 3   reactive_power  3200 non-null   float64
 4   apparent_power  3200 non-null   float64
 5   power_factor    3200 non-null   float64
 6   timestamp       3200 non-null   object 
 7   reliable        3200 non-null   bool   
 8   device_id       3200 non-null   object 
 9   load_type       3200 non-null   object 
 10  probe_id        3200 non-null   object 
 11  frequency       3200 non-null   int64  
 12  energy          3200 non-null   int64  
 13  temperature     3200 non-null   int64  
 14  bulb            3200 non-null   int64  
 15  fan             3200 non-null   int64  
 16  iron            3200 non-null   int64  
 17  labels          3200 non-null   i

Data Preprocessing

In [6]:
x = df.drop(['timestamp','reliable','device_id','load_type','probe_id','frequency','energy','temperature','bulb','fan','iron','labels'],axis = 1)
y = df.labels

In [7]:
x.head()

,voltage,current,active_power,reactive_power,apparent_power,power_factor
0,233.67,0.427060,84.82254,52.56829,99.79122,0.85
1,233.67,0.435269,86.45290,53.57870,101.70930,0.85
2,233.67,0.451382,89.65333,55.56215,105.47450,0.85
3,233.67,0.457015,90.77206,56.25548,106.79070,0.85
4,233.67,0.445174,88.42026,54.79794,104.02380,0.85


In [8]:
y.head()

,labels
0,1
1,1
2,1
3,1
4,1


In [9]:
x.tail()

,voltage,current,active_power,reactive_power,apparent_power,power_factor
3195,233.67,0.535922,106.4445,65.96838,125.2289,0.85
3196,233.67,4.568364,1014.1150,333.32370,1067.4900,0.95
3197,233.67,4.605721,1022.4080,336.04920,1076.2190,0.95
3198,233.67,0.544402,108.1289,67.01224,127.2104,0.85
3199,233.67,0.551037,109.4467,67.82897,128.7608,0.85


In [10]:
x.isnull().sum()

,0
voltage,0
current,0
active_power,0
reactive_power,0
apparent_power,0
power_factor,0


In [11]:
from sklearn.model_selection import train_test_split

x_train,x_,y_train,y_ = train_test_split(x,y,test_size = 0.4,random_state = RANDOM_STATE)
x_cv,x_test,y_cv,y_test = train_test_split(x_,y_,test_size = 0.5,random_state = RANDOM_STATE)
del x_,y_

In [12]:
x_train.head()

,voltage,current,active_power,reactive_power,apparent_power,power_factor
2648,233.67,0.406913,80.82079,50.08823,95.08328,0.85
474,233.67,0.217090,43.11823,26.72228,50.72733,0.85
2871,233.67,0.545238,108.29500,67.11519,127.40580,0.85
1052,233.67,0.227091,45.10466,27.95335,53.06430,0.85
569,233.67,0.131380,26.09460,16.17198,30.69953,0.85


In [13]:
y_train.head()

,labels
2648,5
474,2
2871,7
1052,4
569,2


In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_cv_scaled = scaler.transform(x_cv)
x_test_scaled = scaler.transform(x_test)

Multiclass Classification

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu,linear
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [17]:
num_classes = len(y_train.unique())

In [33]:
tf.random.set_seed(RANDOM_STATE)
lambdas = [0.0,0.001,0.002,0.01,0.05,0.1,0.2]
models = [None]*len(lambdas)

y_train_adj = y_train - 1

for i in range(len(lambdas)):
  lambda_ = lambdas[i]
  models[i] = Sequential([
      Dense(15,activation = 'relu',kernel_regularizer = tf.keras.regularizers.l2(lambda_)),
      Dense(45,activation = 'relu',kernel_regularizer = tf.keras.regularizers.l2(lambda_)),
      Dense(38,activation = 'relu',kernel_regularizer = tf.keras.regularizers.l2(lambda_)),
      Dense(num_classes,activation='linear')
  ])

  models[i].compile(
      loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),

  )

  models[i].fit(
      x_train_scaled,y_train_adj,
      epochs = 200
  )

  print(f'Lambda = {lambda_}')


Epoch 1/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1.8932
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.6572
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3443
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1724
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0857
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0414
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0173
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0038
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9942
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9873
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9818
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9777
Epoch 13/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9737
Epoch 14/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9706
Epoch 15/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9674
Epoc

In [34]:
from sklearn.metrics import accuracy_score

y_cv_adj=y_cv-1

cv_accuracies = []
train_accuracies = []

for i in range(len(models)):
    train_logits = models[i].predict(x_train_scaled)
    train_softmax_outputs = tf.nn.softmax(train_logits).numpy()
    predicted_labels = np.argmax(train_softmax_outputs,axis=1)
    accuracy_train = accuracy_score(y_train_adj, predicted_labels)
    train_accuracies.append(accuracy_train)

    cv_logits = models[i].predict(x_cv_scaled)
    cv_softmax_outputs = tf.nn.softmax(cv_logits).numpy()
    predicted_labels = np.argmax(cv_softmax_outputs,axis=1)
    accuracy_cv = accuracy_score(y_cv_adj, predicted_labels)
    cv_accuracies.append(accuracy_cv)

print(train_accuracies)
print(cv_accuracies)

60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
[0.5848958333333333, 0.5755208333333334, 0.55, 0.5458333333333333, 0.5067708333333333, 0.4791666666666667, 0.15572916666666667]
[0.609375, 0.578125, 0.5515625, 0.553125, 0.5171875, 0.484375, 0.134375]
